In [1]:
!pip install beautifulsoup4

In [3]:
!pip install selenium

In [4]:
#https://builtin.com/articles/selenium-web-scraping
#https://brightdata.com/blog/how-tos/using-selenium-for-web-scraping#:~:text=Before%20you%20start%20web%20scraping,Chrome%2C%20Firefox%2C%20and%20Edge.

#Import selenium and other required packages for the scrape
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [5]:
#Set up the chrome driver options
options = Options()
options.add_argument("--headless")

#Set up the driver
service = Service("C:/Users/Jane Barraclough/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [6]:
#Get the url
url = "https://howstat.com/Cricket/Statistics/Matches/MatchListCountry.asp?A=SAF&B=XXX&C=XXX"
driver.get(url)

#Get the full HTML and close the browser
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.quit()

In [7]:
#I know I need table 5 for my analysis from the previous scrape attempt
tables = soup.find_all('table')
match_table = tables[5]
rows = match_table.find_all('tr')

In [8]:
#Need to confirm the number of rows matches the matches played
print(len(rows))

473


In [9]:
#Get the column names
header_row = rows[0]
headers = [cell.text.strip() for cell in header_row.find_all(['th', 'td'])]
print(headers)

['', 'Date', 'Series', 'Ground', 'Result']


In [10]:
#Extract all match data rows
data_rows = []

for row in rows[1:]:
    cells = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
    if cells:
        data_rows.append(cells)

print(data_rows[0:4])

[['1', '12/03/1889', '1888-1889 South Africa v. England', "St George's Park", 'England won by 8 wickets'], ['2', '25/03/1889', '1888-1889 South Africa v. England', 'Newlands', 'England won by an innings and 202 runs'], ['3', '19/03/1892', '1891-1892 South Africa v. England', 'Newlands', 'England won by an innings and 189 runs'], ['4', '13/02/1896', '1895-1896 South Africa v. England', "St George's Park", 'England won by 288 runs']]


In [11]:
df = pd.DataFrame(data_rows, columns=headers)
df.columns = ['Test Number', 'Date', 'Series', 'Ground', 'Result']
df.head()

,Test Number,Date,Series,Ground,Result
0,1,12/03/1889,1888-1889 South Africa v. England,St George's Park,England won by 8 wickets
1,2,25/03/1889,1888-1889 South Africa v. England,Newlands,England won by an innings and 202 runs
2,3,19/03/1892,1891-1892 South Africa v. England,Newlands,England won by an innings and 189 runs
3,4,13/02/1896,1895-1896 South Africa v. England,St George's Park,England won by 288 runs
4,5,02/03/1896,1895-1896 South Africa v. England,Old Wanderers,England won by an innings and 197 runs


In [12]:
#Download to a CSV file
df.to_csv('../data/south_africa_match_results.csv', index=False)